<h1 style='background:#00091c; border:0;'><center style='color:#00cdff'>Extração de Entidade Nomeada com BioBERTpt</center></h1> 

O BioBERTpt - Portuguese Clinical and Biomedical BERT é um modelo baseado no BERT para língua portuguesa e treinado em notas clínicas e literatura biomédica.

Link: https://huggingface.co/pucpr

O objeitvo desse notebook é extrair entidades que possam auxiliar na separação de diagnósticos dos laudos de exames radiológicos. Foram usados 5 modelos pré-treinados do BioBERTpt: Diagnostic, Disease, Sign, Disorder e Finding.

O algoritmo executa os seguintes passos:

    1. Itera sobre cada modelo (são modelos carregados separadamente)
    2. Codifica o texto de acordo com BioBERTpt
    3. Gera os input_ids de cada palavra dos textos codificados
    4. Gera a label (Entidade) para cada input_id 
    5. Mapeia a label para cada palavra (Ex: 'enfisema': 'B-Disorder') de acordo com o nome dado pelo modelo
    6. Monta um Dataframe indicando o Diagnostic, Disease, Sign, Disorder e Finding para cada laudo do conjunto de dados

In [13]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForTokenClassification, BertConfig
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize  
from transformers import pipeline
import pandas as pd
import re

In [14]:
MODELS_DIR = [
    r"pucpr/clinicalnerpt-diagnostic",
    r"pucpr/clinicalnerpt-disease",           
    r"pucpr/clinicalnerpt-sign",
    r"pucpr/clinicalnerpt-disorder",             
    r"pucpr/clinicalnerpt-finding"   
]

In [64]:
dataset = pd.read_csv('new_dataset.csv') # novo dataset com textos menores que 2000 (o limite que o modelo aceita)
dataset

,docid,modalidade,tipo_exame,laudo_completo
0,189988,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
1,376443,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,realizados cortes tomográficos axiais com técn...
2,1843933,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,tecnica: \n o estudo tomográfico computadoriza...
3,454421,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
4,61595,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,realizada tomografia computadorizada do tórax ...
...,...,...,...,...
4961,948854,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,tc do tórax \n técnica: obtidas imagens axiais...
4962,1668375,CR,RADIOGRAFIA DO TÓRAX,partes moles sem particularidades. \n sinais d...
4963,527535,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
4964,131674,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,infiltrado intersticio-alveolar com broncogram...


In [58]:
def clean_text(sent):
    sent = re.sub("\s+", " ", sent)  
    sent = re.sub("\'", "", sent)  
    sent = re.sub("  ", " ", sent)  
    sent = re.sub("\(", "", sent)  
    sent = re.sub("\)", "", sent)  
    
    return sent

In [63]:
class EntityExtraction:
    def __init__(self, sentences, dataframe, models_name):
        self.sentences = sentences
        self.models_name = models_name
        self. dataframe = dataframe
        self.all_data_entities = []
        self.all_data = []

    def extract_entity(self):
        count = 0
        for sentence in self.sentences:
            try:
                report = clean_text(sentence)
                count+=1
                for model_name in self.models_name:
                    model = BertForTokenClassification.from_pretrained(model_name)

                    tokenizer = BertTokenizer.from_pretrained("pucpr/biobertpt-all", do_lower_case=True)

                    config = BertConfig.from_pretrained(model_name)
                    dic_label=config.id2label

                    tokenized_sentence = tokenizer.encode(report, truncation=True)

                    input_ids = torch.tensor([tokenized_sentence])#.cuda()

                    with torch.no_grad():
                        output = model(input_ids)

                    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

                    idx = 1
                    enc = [tokenizer.encode(x, add_special_tokens=False) for x in report.split()]
                    desired_output = []

                    for token in enc:
                        tokenoutput = []
                        for ids in token:
                            tokenoutput.append(idx)
                            idx +=1
                        desired_output.append(tokenoutput)

                    word_id_list = []
                    labels = []


                    for word_id in desired_output:
                        idx = word_id[-1]
                        labels.append(label_indices[0][idx])
                        entities = [dic_label.get(e) for e in labels]


                    if definitive_entity == []:
                        definitive_entity.extend(entities)

                    for v in entities:
                        if not 'O' in v:
                            indices = [i for i, x in enumerate(entities) if x == v]
                            if len(indices) < 2:
                                definitive_entity[indices[0]] = v
                            else:
                                for o in indices:
                                    definitive_entity[o] = v
                self.all_data_entities.append(definitive_entity.copy())
                definitive_entity.clear()
                
            except:
                print('Sentence is too big!')
                pass
    
    def build_entity_mapping(self):
        diagnostics = []
        fidings = []
        diseases = []
        signs = []
        disorders = []
        data = []
        
        for r in range(len(self.sentences)):
            text = sent_to_words(self.sentences[r])
            text = text.split()

            ent = self.all_data_entities[r]

            for idxmywords in range(len(text)):
                if 'DiagnosticProcedure' in ent[idxmywords]:
                    diagnostics.append(text[idxmywords])
                elif 'Disorder' in ent[idxmywords]:
                    disorders.append(text[idxmywords])
                elif 'Finding' in ent[idxmywords]:
                    fidings.append(text[idxmywords])
                elif 'Sign' in ent[idxmywords]:
                    signs.append(text[idxmywords])
                elif 'Disease' in ent[idxmywords]:
                    diseases.append(text[idxmywords])
                else:
                    pass

            data.append([
                        self.dataframe['docid'][r],
                        self.sentences[r],
                        ", ".join(diagnostics.copy()),
                        ", ".join(disorders.copy()),
                        ", ".join(fidings.copy()),
                        ", ".join(signs.copy()),
                        ", ".join(diseases.copy())
                        ])
            diagnostics.clear()
            disorders.clear()
            fidings.clear()
            signs.clear()
            diseases.clear()
        self.all_data.extend(data.copy())
        data.clear()
    
    def build_dataframe(self):
        df = pd.DataFrame(self.all_data, 
                          columns=["docid", "laudo_completo", "DiagnosticProcedure", "Disorder", "Finding", "Sign", "Disease"])
        return df

In [61]:
my_entities = EntityExtraction(df['laudo_completo'].values[:10], df, MODELS_DIR)

In [62]:
my_entities.extract_entity()
my_entities.build_entity_mapping()
final_df = my_entities.build_dataframe()
final_df

,docid,laudo_completo,DiagnosticProcedure,Disorder,Finding,Sign,Disease
0,189988,o estudo tomográfico computadorizado do tórax ...,"estudo, tomográfico, computadorizado, do, tóra...",massas,"coeficientes, de, atenuação, calibre, e, aspec...",,
1,376443,realizados cortes tomográficos axiais com técn...,"cortes, tomográficos, axiais",,"e, e, morfologia",,
2,1843933,tecnica: \n o estudo tomográfico computadoriza...,"estudo, tomográfico, computadorizado, do, tóra...","enfisema, centrolobular, aspecto, inflamatório","calibre, e, aspecto, brônquios-fonte, de, calibre",,
3,454421,o estudo tomográfico computadorizado do tórax ...,"estudo, tomográfico, computadorizado, do, tóra...",massas,"calibre, e, aspecto, contornos, e, dimensões, ...",,
4,61595,realizada tomografia computadorizada do tórax ...,"tomografia, computadorizada, do, tórax","enfisema, do, espaço, pleural","e, e, e, e",,
5,670634,o estudo tomográfico computadorizado do tórax ...,"estudo, tomográfico, computadorizado, do, tóra...","fibrocicatriciais, enfisema, processo, inflama...","calibre, estruturas, hilares, pérvios, calibre",,
6,563100,o estudo tomográfico computadorizado do tórax ...,"estudo, tomográfico, computadorizado, do, tóra...",,"coeficientes, de, atenuação, calibre, e, aspec...",,
7,663773,exame laudado no sistema de telerradiologia. \...,"sistema, técnica, cortes, tomográficos, comput...","espondilose, processo, inflamatório, infeccios...",,,
8,815751,aquisição com cortes axiais e reformatações mu...,pulmonar,"opacidades, em, fosco, bandas","vidro, pérvios, calibre, normal",,
9,375232,o estudo tomográfico computadorizado do tórax ...,"estudo, tomográfico, computadorizado, do, tóra...","alterações, crônicas, de, processo, granulomat...","alongada, coeficientes, de, atenuação, de, cal...",,
